In [1]:
import numpy as np 
import math

# Dados de entrada

In [2]:
matriz_posicao = [[0.280, 0, 0.300]]
matriz_forca  = [[0, 16e3, 12e3]]
F = 20e3
matriz_momento = ''
d= 0.08
tensao_ruptura = 1000e6
lim_esc = 800e6
aco = 'forjado'
carga = 'axial'
conf = 98
q = 1
kt = 1
n = 2
temp = 80

## Fator de concentração (Kp)

In [3]:
def kp(q,kt):
  kp = 1 + (q * (kt - 1))
  return kp

In [4]:
kp(q,kt)

1

## Limite de resistência a fadiga (Cálculo dos K's)

### Acabamento Superficial (Ka) 

In [5]:
def ka(aco):
  if aco == 'forjado':
    a = 272
    b = -0.995
    ka = a * (tensao_ruptura ** b)
  elif aco == 'laminado_quente':
      a = 57.7
      b = -0.718
      ka = a * (tensao_ruptura ** b)
  elif aco == 'usinado_ou_laminado_frio':
      a = 4.51
      b = -0.265
      ka = a * (tensao_ruptura ** b)
  elif aco == 'retificado':
      a = 1.58
      b = -0.085
      ka = a * (tensao_ruptura ** b)
  elif aco == 'espelhado':
      a = 1
      b = 0
      ka = a * (tensao_ruptura ** b)
  return ka

In [6]:
if ka(aco) > 1:
  ka = 1
else:
  ka = ka(aco)

### Tamanho da Peça (Kb)

In [7]:
def kb(d):
  if carga == 'axial':
    if d == '':
      kb = 0.85
    else:
      kb = 1
  else:
    if d == '':
      kb = 0.85

    elif 2.79 <=  d <= 51:
        kb = (d / 7.62)**-0.107

    elif 51 < d <= 254:
        kb = 1.51 * (d**-0.157)
  
  return kb

In [8]:
if kb(d) > 1:
  kb = 1
else:
  kb = kb(d)

### Confiabilidade (Kc)

In [9]:
def kc(conf):
  if conf == 50:
    kc = 1
    
  elif conf == 90:
      kc = 0.897

  elif conf == 95:
      kc = 0.868

  elif conf == 98:
      kc = 0.836

  elif conf == 99:
      kc = 0.814

  elif conf == 99.99:
      kc = 0.702

  elif conf == 99.9999:
      kc = 0.620

  elif conf == 99.99999:
      kc = 0.584

  return kc

In [10]:
if kc(conf) > 1:
  kc = 1
else:
  kc = kc(conf)

### Temperatura (Kd)

In [11]:
def kd(temp):
  kd = 0.9877 + (0.6507 * (10**-3) * temp) - (0.3414 * (10**-5) * (temp**2)) + (0.5621 * (10**-8) * (temp**3)) - (6.246 * (10**-12) * (temp**4))

  return kd

In [12]:
if kd(temp) > 1:
  kd = 1
else:
  kd = kd(temp)

### Fatores diversos (Kf)

In [13]:
kf = 1

In [14]:
#determinação dos limites de resistência para o cálculo das tensões
lim_normal_res = ka* kb * kc * kd * 0.85 * kf * 0.5 * tensao_ruptura
lim_cisalhante_res = ka * kb * kc * kd * 0.577 * kf * 0.5 * tensao_ruptura

In [15]:
print(lim_normal_res)
print(lim_cisalhante_res)

107.19242882666288
72.76474286233469


## Produto Vetorial

In [16]:
def prod_vet(matriz_posicao, matriz_forca, matriz_momento):
  i = 0
  a = []

  while i < len(matriz_posicao):
      prod = np.cross(matriz_posicao[i], matriz_forca[i])
      prod = a.append(prod)
      prod = sum(a)
      i = i + 1

  if matriz_momento == '':
    matriz_forca = np.sum(matriz_forca, axis=0)
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    print(f'Produto Vetorial: {prod}')
    print(f'F = {matriz_forca[0]} i + {matriz_forca[1]} j + {matriz_forca[2]} k')
    print(f'M = {prod[0]} i + {prod[1]} j + {prod[2]} k ')
    print(f'Momento torçor [T]: {t}')
    print(f'Momento fletor [M]: {m}')
  else:
    matriz_forca = np.sum(matriz_forca, axis=0)
    matriz_momento = np.sum(matriz_momento, axis=0)
    prod = prod + matriz_momento
    t = prod[2]
    m = math.sqrt(((prod[0])**2) + ((prod[1])**2))
    print(f'Produto Vetorial: {prod}')
    print(f'F = {matriz_forca[0]} i + {matriz_forca[1]} j + {matriz_forca[2]} k')
    print(f'M = {prod[0]} i + {prod[1]} j + {prod[2]} k ')
    print(f'Momento torçor [T]: {t}')
    print(f'Momento fletor [M]: {m}')

 
  return m, t


In [17]:
x = prod_vet(matriz_posicao, matriz_forca, matriz_momento)

Produto Vetorial: [-4800. -3360.  4480.]
F = 0.0 i + 16000.0 j + 12000.0 k
M = -4800.0 i + -3360.0000000000005 j + 4480.0 k 
Momento torçor [T]: 4480.0
Momento fletor [M]: 5859.146695552177


In [18]:
#determinar t (momento torçor) e m(momento fletor) para calcular as tensões logo abaixo
t = x[1]
m = x[0]
print(t)
print(m)

4480.0
5859.146695552177


## Cálculo das Tensões

In [19]:
#Cálculo das tensões

if d == '':
  tensao_cisalhante_esc = 0.577 * lim_esc

  def tensao_normal_max(d):
    return (kp(1,1) * ((32 * m) / (3.14 * d**3))) + (kp(1,1) * ((4 * F) / (3.14 * d**2)))

  tensao_normal_min = 0

  def tensao_cisalhante_max(d):
    return (kp(1,1) * ((16 * t) / (3.14 * d**3)))

  tensao_cisalhante_min = 0

  def tensao_normal_var(d):
    return (tensao_normal_max(d) - tensao_normal_min) / 2

  def tensao_normal_media(d):
    return (tensao_normal_max(d) + tensao_normal_min) / 2

  def tensao_cisalhante_media(d):
    return (tensao_cisalhante_max(d) + tensao_cisalhante_min) / 2

  def tensao_cisalhante_var(d):
    return (tensao_cisalhante_max(d) - tensao_cisalhante_min) / 2

  def tensao_normal(d):
    return tensao_normal_media(d) + (tensao_normal_var(d) * (lim_esc/lim_normal_res))

  def tensao_cisalhante(d):
    return tensao_cisalhante_media(d) + (tensao_cisalhante_var(d) * (tensao_cisalhante_esc/lim_cisalhante_res))
  
else: 
  tensao_cisalhante_esc = 0.577 * lim_esc

  tensao_normal_max = (kp(1,1) * ((32 * m) / (3.14 * (d**3)))) + (kp(1,1) * ((4 * F) / (3.14 * (d**2))))
  tensao_normal_min = 0
  tensao_cisalhante_max = kp(1,1) * ((16 * t) / (3.14 * (d**3)))
  tensao_cisalhante_min = 0

  tensao_normal_var = (tensao_normal_max - tensao_normal_min) / 2
  tensao_normal_media = (tensao_normal_max + tensao_normal_min) / 2

  tensao_cisalhante_media = (tensao_cisalhante_max + tensao_cisalhante_min) / 2
  tensao_cisalhante_var = (tensao_cisalhante_max - tensao_cisalhante_min) / 2

## Método de Newton 

In [20]:
if d == '':
  d0 = math.sqrt((4*F) / lim_esc)
  print(d0)
else:
  print('.')

.


In [21]:
def f(d):
  return (n * (tensao_normal(d)**2 + (3* tensao_cisalhante(d)**2))**(1/2))/lim_esc

In [22]:
def flin(f, d):  
  if d == 0:
    d = 1
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  else:
    h = d / 5471
    flin = (1/(12*h)) * ( f(d- 2*h) - 8*f(d - h) + 8*f(d+h) - f(d + 2*h) )
  return flin

In [23]:
def newton(f, d0, epsilon, maxIter=20):
    if abs(f(d0)) <= epsilon:
      return (d0)

    print("k\t d0\t\t f(d0)") #cabeçalho da tabela que retornará com os valores embaixo. \t é o espaçamento que dou para ficar visual.
    
    k=1
    while k <= maxIter:
      d1 = d0 - f(d0)/flin(f,d0)
        
      print("%d\t%e\t%e" % (k, d1,f(d1)))

      if abs(f(d1)) <= epsilon:
        return d1      
      
      d0 = d1
      k = k + 1
    
    print('ERRO: Número máximo de iterações atingido')
    
    return d1  


## Critérios de Resistência a Fadiga

### Sorderberg

In [24]:
def sorderberg(tensao_normal_media,tensao_normal_var,lim_esc,lim_normal_res,tensao_cisalhante_media,tensao_cisalhante_var,tensao_cisalhante_esc,lim_cisalhante_res):
  tensao_normal_sorderberg = tensao_normal_media + (tensao_normal_var * (lim_esc / lim_normal_res))
  tensao_cisalhante_sorderberg = tensao_cisalhante_media + (tensao_cisalhante_var * (tensao_cisalhante_esc / lim_cisalhante_res))
  fat_seg_sorderberg = lim_esc / ((tensao_normal_sorderberg**2) + (3 * (tensao_cisalhante_sorderberg**2)))**(1/2)

  return print(f'Tensão normal sorderberg: {tensao_normal_sorderberg} \nTensão cisalhante sorderberg: {tensao_cisalhante_sorderberg} \nN sorderberg: {fat_seg_sorderberg}')
  

### Modificado de Goodman

In [25]:
def mod_good(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media):
  tensao_normal_mod_god = lim_esc * ((tensao_normal_var/lim_normal_res) + (tensao_normal_media/tensao_ruptura))
  tensao_cisalhante_mod_god = tensao_cisalhante_esc * ((tensao_cisalhante_var/lim_cisalhante_res) + (tensao_cisalhante_media/(tensao_ruptura*0.8)))
  fat_seg_mod_god = lim_esc / (((tensao_normal_mod_god**2) + 3 * (tensao_cisalhante_mod_god**2))**(1/2))

  return print(f'Tensão normal modificado de goodman: {tensao_normal_mod_god} \nTensão cisalhante modificado de goodman: {tensao_cisalhante_mod_god} \nN modificado de goodman: {fat_seg_mod_god}')

### Critério de Gerber

In [26]:
def gerber(a1,lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura):
  a1 = 1
  b1 = (lim_esc * tensao_normal_var) / lim_normal_res
  c1 = ((lim_esc * tensao_normal_media) / tensao_ruptura)**2

  if(a1==0):
      print('a=0, não é equação do segundo grau')
  else:
      delta1 = b1*b1 - (4*a1*(-c1))

      if delta1<0:
          print('Delta menor que 0. Raízes imaginárias')
      elif delta1==0:
          tensao_normal_gerber = -b1 / (2*a1)
          print('Delta=0 , raiz = ',tensao_normal_gerber)
      else:
          tensao_normal_gerber1 = (-b1 + math.sqrt(delta1) ) / (2*a1)
          tensao_normal_gerber2 = (-b1 - math.sqrt(delta1) ) / (2*a1)
          print(f'Tensão normal de gerber: {tensao_normal_gerber1}' )

  a2 = 1
  b2 = (tensao_cisalhante_esc * tensao_cisalhante_var) / lim_cisalhante_res
  c2 = ((tensao_cisalhante_esc * tensao_cisalhante_media) / (tensao_ruptura * 0.8))**2 

  if(a2==0):
      print('a=0, não é equação do segundo grau')
  else:
      delta2 = b2*b2 - (4*a2*(-c2))

      if delta2<0:
          print('Delta menor que 0. Raízes imaginárias')
      elif delta2==0:
          tensao_cisalhante_gerber = -b2 / (2*a2)
          print('Delta=0 , raiz = ',tensao_cisalhante_gerber)
      else:
          tensao_cisalhante_gerber1 = (-b2 + math.sqrt(delta2) ) / (2*a2)
          tensao_cisalhante_gerber2 = (-b2 - math.sqrt(delta2) ) / (2*a2)
          print(f'Tensão cisalhante de gerber: {tensao_cisalhante_gerber1}' )

  fat_seg_gerber = lim_esc / (((tensao_normal_gerber2 **2) + 3 * (tensao_cisalhante_gerber2 **2))**(1/2))
  return print(f'N gerber: {fat_seg_gerber}')

### ASME

In [27]:
def asme(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media):
  tensao_normal_asme = math.sqrt((((lim_esc * tensao_normal_var)/lim_normal_res)**2) + ((tensao_normal_media)**2))
  tensao_cisalhante_asme = math.sqrt((((tensao_cisalhante_esc * tensao_cisalhante_var)/lim_cisalhante_res)**2) + ((tensao_cisalhante_media)**2))
  fat_seg_asme = lim_esc/math.sqrt((tensao_normal_asme**2) + 3*(tensao_cisalhante_asme**2))

  print(f'Tensão normal ASME: {tensao_normal_asme}')
  print(f'Tensão cisalhante ASME: {tensao_cisalhante_asme}')
  print(f'N ASME: {fat_seg_asme}')
  return 

### Escoamento Ampliado

In [28]:
def esc_amp(tensao_normal_var,tensao_normal_media,tensao_cisalhante_var,tensao_cisalhante_media,lim_esc):
  tensao_normal_escamp = tensao_normal_var + tensao_normal_media
  tensao_cisalhante_escamp = tensao_cisalhante_var + tensao_cisalhante_media
  fat_seg_escamp = lim_esc/((tensao_normal_escamp**2) + 3*(tensao_cisalhante_escamp**2))**(1/2)

  print(f'Tensão normal escoamento ampliado: {tensao_normal_escamp}')
  print(f'Tensão cisalhante escoamento ampliado: {tensao_cisalhante_escamp}')
  print(f'N escoamento ampliado {fat_seg_escamp}')

  return

# **Código Principal**

In [29]:
if d == '':
  kp (1,1)

  print(f'Limite normal de resistência: {lim_normal_res}')
  print(f'Limite cisalhante de resistência: {lim_cisalhante_res}')

  print('-'*50)

  prod_vet([[0.28, 0, 0.3]],[[0, 16, 12]],'')

  print('-'*50)

  print(f'Diâmetro inicial: {d0}')

  raiz = newton(f,d0,10**-8,5) 

  print(raiz)

else:
  kp (1,1)

  # lim_normal_res = ka('forjado') * kb('') * kc(99) * kd(80) * ke('axial') * kf * (0.5 * tensao_ruptura)
  # lim_cisalhante_res = ka('forjado') * kb('') * kc(99) * kd(80) * ke('torcao') * kf * (0.5 * tensao_ruptura)

  print(f'Limite normal de resistência: {lim_normal_res}')
  print(f'Limite cisalhante de resistência: {lim_cisalhante_res}')

  print('-'*50)

  prod_vet([[0.28, 0, 0.3]],[[0, 16, 12]],'')

  print('-'*50)

  print(f'Tensão cisalhante de escoamento: {(tensao_cisalhante_esc* 1e-6):.2f}')
  print(f'Tensão normal mínima: {(tensao_normal_min* 1e-6):.2f} MPa')    
  print(f'Tensão normal máxima: {(tensao_normal_max* 1e-6):.2f} MPa')    
  print(f'Tensão cisalhante máxima: {(tensao_cisalhante_max* 1e-6):.2f} MPa')    
  print(f'Tensão cisalhante mínima: {(tensao_cisalhante_min* 1e-6):.2f} MPa')

  print('-'*50)

  print(f'Tensão normal variável: {(tensao_normal_var* 1e-6)}')
  print(f'Tensão normal média: {(tensao_normal_media* 1e-6)}')
  print(f'Tensão cisalhante média: {(tensao_cisalhante_media* 1e-6)}')
  print(f'Tensão cisalhante variável: {(tensao_cisalhante_var* 1e-6)}')

  print('-'*50)

  sorderberg(tensao_normal_media,tensao_normal_var,lim_esc,lim_normal_res,tensao_cisalhante_media,tensao_cisalhante_var,tensao_cisalhante_esc,lim_cisalhante_res)

  print('-'*50)

  mod_good(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media)

  print('-'*50)

  gerber(1,lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_ruptura)

  print('-'*50)

  asme(lim_esc,tensao_normal_var,lim_normal_res,tensao_normal_media,tensao_cisalhante_esc,tensao_cisalhante_var,lim_cisalhante_res,tensao_cisalhante_media)

  print('-'*50)

  esc_amp(tensao_normal_var,tensao_normal_media,tensao_cisalhante_var,tensao_cisalhante_media,lim_esc)


Limite normal de resistência: 107.19242882666288
Limite cisalhante de resistência: 72.76474286233469
--------------------------------------------------
Produto Vetorial: [-4.8  -3.36  4.48]
F = 0 i + 16 j + 12 k
M = -4.8 i + -3.3600000000000003 j + 4.48 k 
Momento torçor [T]: 4.48
Momento fletor [M]: 5.859146695552178
--------------------------------------------------
Tensão cisalhante de escoamento: 461.60
Tensão normal mínima: 0.00 MPa
Tensão normal máxima: 120.60 MPa
Tensão cisalhante máxima: 44.59 MPa
Tensão cisalhante mínima: 0.00 MPa
--------------------------------------------------
Tensão normal variável: 60.30201727261321
Tensão normal média: 60.30201727261321
Tensão cisalhante média: 22.292993630573243
Tensão cisalhante variável: 22.292993630573243
--------------------------------------------------
Tensão normal sorderberg: 450046899860997.6 
Tensão cisalhante sorderberg: 141420790855885.97 
N sorderberg: 1.5613160123921195e-06
------------------------------------------------

# Tabela Auxiliar

## Limite de resistência



<img src='tabela.png'>